In [1]:
"""Purpose: to parse void catalogue and save it as a new text file
relavent to current work."""

'Purpose: to parse void catalogue and save it as a new text file\nrelavent to current work.'

In [2]:
import pandas as pd
import os
import re


In [3]:
from astropy.table import QTable
from astropy.io import fits
from astropy import units as u


In [4]:
# Void catalog file in text format expected to be in current directory
FN = "apjaa508et1_mrt.txt"

n_junkrows = 35 # number of junk rows above data

# Column names as they come from the catalog
colnames = ['SamplePartA', 'SamplePartB', 'ID', 'RAdeg', 'DEdeg', 'z', 'NGal', 'V', 'Reff', 'nmin', 'delmin', 'r', 'P', 'Dbound']

# Columns we want
keepCols = ['ID', 'RAdeg', 'DEdeg', 'z', 'Reff']


In [5]:
keepCols[1]

'RAdeg'

In [6]:
data = pd.read_table('apjaa508et1_mrt.txt', skiprows=n_junkrows, header=None, names=colnames,
                     delim_whitespace=True, index_col=False)

parsed_data = data[keepCols]


In [7]:
parsed_data.loc[:, "DR"] = 12 # Keep Track of what DR the data comes from
parsed_data.loc[:, 'label'] = 'Mao' # Author of the 'other' void catalog

/var/folders/pj/_3cv5_xd2sz143_b3_wm1xk40000gn/T/ipykernel_5044/1011862754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parsed_data.loc[:, "DR"] = 12 # Keep Track of what DR the data comes from
/var/folders/pj/_3cv5_xd2sz143_b3_wm1xk40000gn/T/ipykernel_5044/1011862754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parsed_data.loc[:, 'label'] = 'Mao' # Author of the 'other' void catalog


In [8]:
main_folder = './SDSS Data Used/'
# grab everything in betwen dr[any digit][0 or 1 '0'] and .dat.out
pattern = re.compile(r'dr\d0?(.*?)\.dat\.out')
colnames = [keepCols[1], keepCols[2], keepCols[3], keepCols[4], keepCols[0]]
# 'RAdeg', 'DEdeg', 'z', 'Reff' , 'ID'


dfs = {}
for root, dirs, files in os.walk(main_folder):
    if '.DS_Store' in dirs:
        dirs.remove('.DS_Store')
    if '.DS_Store' in files:
        files.remove('.DS_Store')


    for file in files:
        match = pattern.search(file)
        label = match.group(1) # category name
        dir_name = os.path.basename(root)
        # print(dir_name)
        file_path = os.path.join(root, file)
        # print(file)
        temp = pd.read_table(file_path, 
                            delim_whitespace=True, 
                            header=None,
                            skiprows=1,
                            names=colnames)
        temp['label'] = label
        temp['DR'] = int(dir_name)
        dfs[label] = temp

        
        

In [9]:
temp = [parsed_data]
for key in dfs:
    void_df = pd.concat(temp)
    temp = [void_df, dfs[key]]

In [10]:
# void_df # public void catalog adds ~ 5,000 voids!

In [11]:
void_df.to_csv('void_data.txt') # Save data

# Graveyard

In [12]:
# # DEFUNCT. NO LONGER USING FITS FILE. i couldnt figure it out :()
# # Creat the fits file. 
# column_names = list(void_df.columns.values)

# column_names
# voids = void_df.to_numpy()

# voids[:,0]

# # WARNING
# # Reff h unit removed, because units are Mpc/h where h is "dimensionless hubble constant"
# # voids = [voids[:,0] , voids[:,1]*u.deg, voids[:,2]*u.deg, voids[:,3], voids[:,4]*(u.Mpc/u.h)]
# #       ['ID',        'RAdeg',          'DEdeg',          'z',        'Reff']
# voids = [voids[:,0] , voids[:,1]*u.deg, voids[:,2]*u.deg, voids[:,3], voids[:,4]*u.Mpc, voids[:,5], voids[:,6]]# There has to be a better way.


# voids = QTable(voids, names=column_names)

# voids.write('voids.fit', format='fits', overwrite=True)

# # Attempt 2
# # example uses np
# # """voids = parsed_data.to_numpy()
# # cols = fits.ColDefs(voids)""""""


In [13]:
# void_table = QTable.read("voids.fits")

# import ligo.skymap.plot
# from matplotlib import pyplot as plt
# from astropy.coordinates import SkyCoord


# void_coords = SkyCoord(void_table['RAdeg'], void_table['DEdeg'])
# void_coords

# ax = plt.axes(projection='geo aitoff')
# ax.grid()
# # ax.plot(void_coords.ra.deg, void_coords.dec.deg)
# ax.scatter(void_coords.ra.deg, void_coords.dec.deg, transform=ax.get_transform('world'),
#           s=1)
